**Named-Entity_Recognition**

In [ ]:
import spacy

In [ ]:
spacy.__version__

'3.4.4'

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
nlp_lg = spacy.load('en_core_web_lg')

In [ ]:
dc = nlp_lg('Modi Ji is a Prime Minister of India')

In [ ]:
dc.ents[0], type(dc.ents[0])

(Modi Ji, spacy.tokens.span.Span)

In [ ]:
from spacy import displacy
displacy.render(dc, style= 'ent', jupyter=True)

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
trf = spacy.load('en_core_web_trf')

In [ ]:
import spacy_transformers as sp

In [ ]:
text = "Apple reached an all time high stock price of 143 $ in January."
dc = trf(text)

In [ ]:
spacy.displacy.render(dc, style='ent',jupyter=True)

In [ ]:
x = torch.tensor([1, 2, 3, 4]) ##unk
print(torch.unsqueeze(x, 0))

print(torch.unsqueeze(x, -1))
print(torch.unsqueeze(x, -1))

**Custom Named-Entity_Recognition**

In [1]:
import spacy
spacy.__version__

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


'3.4.4'

In [2]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank('en') # load a blank english model
db = DocBin()  # create a DocBin object
db_1 = DocBin()

In [3]:
import json
f = open('Train.json') # load custom annotation file for training
f1 = open('Test.json') # load custom annotation file for test
Train_data = json.load(f)
Test_data = json.load(f1)

In [ ]:
Test_data

In [4]:
for text, annot in tqdm(Train_data['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot['entities']:
    span = doc.char_span(start, end, label=label, alignment_mode = 'contract')
    if span is None:
      print('skipping entities')
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk("./training_data.spacy")  # save the docbin object

100%|██████████| 7/7 [00:00<00:00, 995.19it/s]


In [5]:
for text, annot in tqdm(Test_data['annotations']):
  doc_1 = nlp.make_doc(text)
  ents_1 = []
  for start, end, label in annot['entities']:
    span = doc_1.char_span(start, end, label=label, alignment_mode = 'contract')
    if span is None:
      print('skipping entities')
    else:
      ents_1.append(span)
  doc_1.ents = ents
  db_1.add(doc_1)

db_1.to_disk("./test_data.spacy")  # save the docbin object

100%|██████████| 5/5 [00:00<00:00, 1571.37it/s]


In [8]:
! python -m spacy init fill-config base_config.cfg config_1.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-23 15:28:36.259930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 15:28:36.260114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 15:28:36.260142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [9]:
! python -m spacy train config_1.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./test_data.spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-23 15:28:58.813942: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 15:28:58.814155: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 15:28:58.814189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [11]:
ner_model = spacy.load('/content/model-best')

In [17]:
tex = ner_model("Samsung is a Korean brand")
tex1 = ner_model('my name is manish singh from india')

In [18]:
spacy.displacy.render(tex, style= 'ent',jupyter=True) #
spacy.displacy.render(tex1, style= 'ent',jupyter=True) #

In [63]:
tex.ents, tex1.ents

((Samsung, Korean), (manish singh, india))

In [67]:
tex.ents

AttributeError: ignored

# Sentence Similarity

In [29]:
sentences = ["Three year later, the coffin was still full of Jello",
             "The person box was packed with jelly many dozens of months later",
             "It took him a month to finish the meal",
             "He found a leprechaun in his walnut shell"]

In [ ]:
!pip install transformers

In [30]:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

import torch
from transformers import AutoTokenizer, AutoModel

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [31]:
tokens = {'input_ids': [], 'attention_mask': []}

In [32]:
from transformers.pipelines import NO_TOKENIZER_TASKS
for sentence in sentences:
  n_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True, padding= 'max_length', return_tensors= 'pt')

  tokens['input_ids'].append(n_tokens['input_ids'][0])
  tokens['attention_mask'].append(n_tokens['attention_mask'][0])

In [33]:
len(tokens['input_ids']), tokens['input_ids'][0]

(4,
 tensor([  101,  2093,  2095,  2101,  1010,  1996, 13123,  2001,  2145,  2440,
          1997, 15333,  7174,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     

In [35]:
len(tokens['attention_mask']), tokens['attention_mask'][0]

(4,
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]))

In [36]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [43]:
tokens['input_ids'].shape, tokens['attention_mask'].shape, tokens['input_ids'][:2][:,:20]

(torch.Size([4, 128]),
 torch.Size([4, 128]),
 tensor([[  101,  2093,  2095,  2101,  1010,  1996, 13123,  2001,  2145,  2440,
           1997, 15333,  7174,   102,     0,     0,     0,     0,     0,     0],
         [  101,  1996,  2711,  3482,  2001,  8966,  2007, 20919,  2116,  9877,
           1997,  2706,  2101,   102,     0,     0,     0,     0,     0,     0]]))

In [44]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [45]:
outputs.last_hidden_state.shape

torch.Size([4, 128, 768])

In [46]:
embeddings = outputs.last_hidden_state
embeddings.shape

torch.Size([4, 128, 768])

In [48]:
attention = tokens['attention_mask']
attention.shape

torch.Size([4, 128])

In [49]:
attention.unsqueeze(-1).shape

torch.Size([4, 128, 1])

In [51]:
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()
mask.shape, mask[:2,:5,:10]

(torch.Size([4, 128, 768]), tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]))

In [52]:
mask_embeddings = embeddings * mask
mask_embeddings.shape

torch.Size([4, 128, 768])

In [53]:
summed = torch.sum(mask_embeddings, 1)
summed.shape

torch.Size([4, 768])

In [55]:
counts = torch.clamp(mask.sum(1), min= 1e-9)
counts.shape

torch.Size([4, 768])

In [58]:
mean_pooled = summed/counts
mean_pooled.shape

mean_pooled = mean_pooled.detach().numpy() # convert into numpy 
mean_pooled.shape, mean_pooled

((4, 768), array([[ 0.11072224,  0.8878816 ,  0.0918908 , ...,  0.6946925 ,
          1.8086272 , -0.13047223],
        [-0.4605085 ,  0.8052549 , -0.02001813, ...,  0.1284651 ,
          0.91938347, -0.18270151],
        [-0.19386727,  0.08378778,  0.8740874 , ..., -0.04201014,
          0.836944  , -0.1145752 ],
        [-0.18123649,  1.0047624 , -0.88497436, ...,  0.7034576 ,
          0.16820747, -0.34110814]], dtype=float32))

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

array = cosine_similarity([mean_pooled[0]], mean_pooled[:]) # let's check first sentence 
array
# second sentence looking more similar to second sentence

array([[1.        , 0.7299921 , 0.45121297, 0.5579506 ]], dtype=float32)